# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

Import the dataset with .\mongoimport.exe --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [4]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [6]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [20]:
# confirm that our new database was created
pprint(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [10]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [18]:
# review the collections in our new database
pprint(db.list_collection_names())

['establishments']


In [14]:
# review a document in the establishments collection
db.establishments.find_one()

{'_id': ObjectId('67301a97f7db7d2d4a30fbb3'),
 'FHRSID': 1034540,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DY',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2022-08-17T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 0},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'},
 'RightToReply': '',
 'Distance': 4587.347174863443,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,
  'returncode'

In [16]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [22]:
# Create a dictionary for the new restaurant data
halal_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [24]:
# Insert the new restaurant into the collection
establishments.insert_one(halal_restaurant)

InsertOneResult(ObjectId('67301c961c4a22a17d6c4eb6'), acknowledged=True)

In [32]:
# Check that the new restaurant was inserted
result = list(establishments.find({"BusinessName":"Penang Flavours"}))
pprint(result)

[{'AddressLine1': 'Penang Flavours',
  'AddressLine2': '146A Plumstead Rd',
  'AddressLine3': 'London',
  'AddressLine4': '',
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': '',
  'Distance': 4623.972328074718,
  'LocalAuthorityCode': '511',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'NewRatingPending': True,
  'Phone': '',
  'PostCode': 'SE18 7DY',
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('67301c961c4a22a17d6c4eb6'),
  'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
  'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}]


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [40]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID': 1, 'BusinessType': 1}
result = establishments.find_one(query, fields)
pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('67301a97f7db7d2d4a30fbb4')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [42]:
# Update the new restaurant with the correct BusinessTypeID
BusinessTypeID = result["BusinessTypeID"]
result = db.establishments.update_one(
        {'BusinessName': 'Penang Flavours'},
        {'$set': {'BusinessTypeID': BusinessTypeID}}
    )

In [50]:
# Confirm that the new restaurant was updated
updated_halal_restaurant = db.establishments.find_one(
    {'BusinessName': 'Penang Flavours'},
    {'BusinessName' :1, 'BusinessTypeID': 1, "_id": 0}
)
pprint(updated_halal_restaurant)

{'BusinessName': 'Penang Flavours', 'BusinessTypeID': 1}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [52]:
# Find how many documents have LocalAuthorityName as "Dover"
dover = db.establishments.count_documents({'LocalAuthorityName': 'Dover'})
pprint(dover)

994


In [54]:
# Delete all documents where LocalAuthorityName is "Dover"
dover_result = db.establishments.delete_many({'LocalAuthorityName': 'Dover'})
pprint(dover_result.deleted_count)

994


In [56]:
# Check if any remaining documents include Dover
dover = db.establishments.count_documents({'LocalAuthorityName': 'Dover'})
pprint(dover)

0


In [58]:
# Check that other documents remain with 'find_one'
pprint(db.establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67301a97f7db7d2d4a30fe95'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [60]:
# Change the data type from String to Decimal for longitude and latitude
updated_result = db.establishments.update_many(
    {'geocode.latitude': {'$type': 'string'}, 'geocode.longitude': {'$type': 'string'}},
    [
        {'$set': {
            'geocode.latitude': {'$toDecimal': '$geocode.latitude'},
            'geocode.longitude': {'$toDecimal': '$geocode.longitude'}
        }}
    ]
)

Use `update_many` to convert `RatingValue` to integer numbers.

In [62]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [72]:
# Change the data type from String to Integer for RatingValue
updated_value = db.establishments.update_many(
    {'RatingValue': {'$type': 'string'}},
    [
        {'$set': {
            'RatingValue': {'$toInt': '$RatingValue'}
        }}
    ]
)

In [84]:
# Check that the coordinates and rating value are now numbers
result_check = db.establishments.find_one(
    {},
    {'geocode.latitude': 1, 'geocode.longitude': 1, 'RatingValue': 1, '_id': 0}
)

# Print actual values
pprint(result_check)

# Print value types
print("Latitude:", type(result_check.get('geocode', {}).get('latitude')))
print("Longitude:", type(result_check.get('geocode', {}).get('longitude')))
print("RatingValue:", type(result_check.get('RatingValue')))

{'RatingValue': 4,
 'geocode': {'latitude': Decimal128('51.086058'),
             'longitude': Decimal128('1.196408')}}
Latitude: <class 'bson.decimal128.Decimal128'>
Longitude: <class 'bson.decimal128.Decimal128'>
RatingValue: <class 'int'>
